# Simular Cabo Umbilical

Este Jupyter Notebook (kernel `julia`) prepara e roda alguns casos de simulação. O primeiro passo é importar os módulos e arquivos auxiliares.

In [1]:
include("param_cabo.jl");

import Combinatorics.combinations

In [2]:
@show Threads.nthreads()

Threads.nthreads() = 12


12

## Parâmetros do umbilical

Os condutores e índices correspondentes estão ilustrados na figura seguinte:

<img src="umbilical_condutores.png" alt="umbilical_condutores" width="600"/>

In [3]:
## Geometria do umbilical PT
Lc = 2.5e3  # comprimento total [m]
rc = 1e-3 * ([9.6, 17.054, 18.054, 19.50])  # raios dos cabos SC [m]: condutor central, isolante, blindagem e isolante externo
ra = 1e-3 * ([48, 59, 65])  # raios da armadura [m]: interno, externo, capa
nx = 2  # número de condutores "ativos" em cada cabo SC (single core)

# distância do centro do umbilical ao centro das veias
d01 = rc[4] / cos(deg2rad(30))
d02 = d01
di = [d01, d02, d02]
theta_jk = (2 * pi) / 3  # ângulo entre cabos do umbilical

# permissividades elétricas
epsr_c = 3.31  # condutor central
epsr_b = 2.3  # blindagem, jaqueta da veia
epsr_a = 10  # última camada, que envolve a armadura

# resistividades elétricas [Ohm.m]
rho_c = 1.7241e-8  # condutor central
rho_b = 2.2e-7  # blindagem, jaqueta da veia
rho_a = 2.86e-8  # armadura

mur_a = 300  # permeabilidade magnética relativa da armadura
sig_s = 5.0  # condutividade elétrica do mar [S/m]
eps_s = 81.0  # permissividade elétrica relativa do mar

fases = [1, 3, 5]  # não alterar
blindagens = [2, 4, 6]  # não alterar
armadura = 7  # não alterar
condutores = [fases; blindagens; armadura]
nc1 = length(condutores)  # número de condutores
nc2 = nc1 * 2
nome_condutores = sort(Dict([
    [fases[i] => "fase_$(i)" for i in eachindex(fases)];
    [blindagens[i] => "blindagem_$(i)" for i in eachindex(blindagens)];
    [armadura => "armadura"]
]))
pares_falhas = collect(combinations(condutores, 2))
num_pares_falhas = length(pares_falhas)

21

## Parâmetros da simulação

Por causa da transformada numérica de Laplace utilizada, 20% dos tempos finais serão lixo numérico. Além disto, para minimizar o _zero padding_ (e oscilação numérica) da transformada de Fourier (FFTW), utilize um número de passos no tempo que seja potência de 2.

In [4]:
## Cálculo dos parâmetros e matrizes
## Tempo
dt = 2.5e-9  # passo no tempo [s]
# Usar número nt que seja potência de 2, para diminuir lixo numérico do zero-padding da FFT
nt = 1024 * 98  # número de passos no tempo
tempo = range(0, step=dt, length=nt)
tmax = tempo[end]
tempo_abre = 10e-6  # [s] que a chave desconecta a fonte do cabo
nt_trunc = Int(ceil(nt * 0.80))  # 20% dos tempos finais é lixo numérico
tmax_trunc = tempo[nt_trunc]  # tempo [s] final que será salvo

comprimentos = [0.5, 1.0] .* Lc
arquivo_matrizes = "umbilical_tripolar.h5"
precalc_matrizes(arquivo_matrizes, comprimentos, tmax, nt,
                    rc, ra, rho_a, rho_b, rho_c, epsr_a, epsr_b, epsr_c,
                    mur_a, theta_jk, di, sig_s, eps_s, nx)
#
v_fonte_t = ones(nt)
R_chave = 50.0  # [Ω]
R_curto = 1e-6  # [Ω], tendência de causar instabilidade numérica para valores muito pequenos

1.0e6

## Circuito

O circuito a ser simulado está ilustrado na figura seguinte. Ele consiste de dois segmentos de cabos (parâmetros distribuídos) com dois vetores de resistências: `R_falha_serie` que insere uma resistência em série nos condutores, indivudalmente, no ponto da falha, e `R_falha_shunt` que conecta as fases para suas blindagens e todas as blindagens entre si e estas para a armadura. Um valor negativo de `R_falha_serie[i]` significa sem falha série (curto-circuito) no condutor `i` e `R_falha_shunt[k]` significa que não há falha (resistência infinita) entre o par `k`.

O circuito de injeção consiste numa fonte de tensão em série com uma resistência `R_chave` e uma chave normal fechada que abre no tempo `tempo_abre`. Esse chaveamento é necessário, pois do contrário a fonte de tensão continuaria a forçar um valor no terminal, independentemente do valor de `R_chave`. Depois da chave há um vetor de resistências `R_emissor_shunt` que permanece conectada nos terminais emissor (excitação) durante toda a simulação. Um valor negativo de `R_emissor_shunt[i]` significa que não há resistência conectada no terminal `i`, isto é, equivale a uma resistência infinita.

Podem ser colocados um número arbitrário de segmentos de cabo. A falha é inserida no lado receptor (direita) do segmento correspondente à variável `segmento_falha`. O vetor booleano `aterrar_receptor` controla se o receptor de cada segmento tem as blindagens e armadura aterradas ou não.

<img src="circuito_umbilical.png" alt="circuito_umbilical" width="1000"/>

O bloco seguinte dá a correspondência entre o vetor `R_falha_shunt` e os pares de condutores.

In [8]:
println("Correspondência entre R_falha_shunt e par de condutores:")
for (i, pf) in enumerate(pares_falhas)
    c1 = nome_condutores[pf[1]]
    c2 = nome_condutores[pf[2]]
    println("R_falha_shunt[$(i)] => $(c1) para $(c2)")
end

Correspondência entre R_falha_shunt e par de condutores:
R_falha_shunt[1] => fase_1 para fase_2
R_falha_shunt[2] => fase_1 para fase_3
R_falha_shunt[3] => fase_1 para blindagem_1
R_falha_shunt[4] => fase_1 para blindagem_2
R_falha_shunt[5] => fase_1 para blindagem_3
R_falha_shunt[6] => fase_1 para armadura
R_falha_shunt[7] => fase_2 para fase_3
R_falha_shunt[8] => fase_2 para blindagem_1
R_falha_shunt[9] => fase_2 para blindagem_2
R_falha_shunt[10] => fase_2 para blindagem_3
R_falha_shunt[11] => fase_2 para armadura
R_falha_shunt[12] => fase_3 para blindagem_1
R_falha_shunt[13] => fase_3 para blindagem_2
R_falha_shunt[14] => fase_3 para blindagem_3
R_falha_shunt[15] => fase_3 para armadura
R_falha_shunt[16] => blindagem_1 para blindagem_2
R_falha_shunt[17] => blindagem_1 para blindagem_3
R_falha_shunt[18] => blindagem_1 para armadura
R_falha_shunt[19] => blindagem_2 para blindagem_3
R_falha_shunt[20] => blindagem_2 para armadura
R_falha_shunt[21] => blindagem_3 para armadura


# Rotina Principal

Pré-cálculo das matrizes de admitância nodal $Y_N$ dos segmentos. Resultados salvos em um arquivo HDF5.

In [ ]:
comprimentos = [Lc]
precalc_matrizes("umbilical_tripolar.h5", comprimentos, tmax, nt,
                 rc, ra, rho_a, rho_b, rho_c, epsr_a, epsr_b, epsr_c,
                 mur_a, theta_jk, di, sig_s, eps_s, nx)

Carregar as matrizes $Y_N$ dos segmentos da simulação.

In [ ]:
comprimentos = [Lc/2, Lc/2]
num_segmentos = length(comprimentos)
sk, _ = laplace(ones(nt), tmax, nt)
nf = length(sk)  # número de frequências
yn_segmentos = zeros(ComplexF64, (nc2, nc2, nf, num_segmentos))
for k = 1:num_segmentos
    L1 = comprimentos[k]
    nome = "yn_" * string(round(L1, sigdigits=2))
    yn_segmentos[:,:,:,k] .= reshape(h5read(arquivo_matrizes, nome), (nc2, nc2, nf))
end

In [ ]:
R_emissor_shunt = fill(-1.0, nc1)  # tudo negativo: resistência infinita
R_falha_shunt = fill(-1.0, num_pares_falhas)  # tudo negativo: resistência infinita
R_falha_serie = fill(-1.0, nc1)  # tudo negativo: resistência nula
segmento_falha = 1
aterrar_receptor = [false, true]
vout_t = simular_cabo(yn_segmentos, R_curto, R_chave, R_emissor_shunt,
                      R_falha_shunt, R_falha_serie, segmento_falha,
                      pares_falhas, terminal_fonte, fases, blindagens,
                      armadura, v_fonte_t, tempo_abre, aterrar_receptor)
arquivo_resultados = "teste/falha_ambas_falhas_t$(terminal_fonte).csv"
salvar_resultados(arquivo_resultados, vout_t, v_fonte_t, tempo, tempo_abre,
                  comprimentos, segmento_falha, pares_falhas, terminal_fonte,
                  R_emissor_shunt, R_falha_serie, R_falha_shunt, R_chave,
                  nt_trunc, nome_condutores, aterrar_receptor)